In [13]:
## Code base used as reference: https://python.langchain.com/docs/integrations/providers/cohere/
import os 
from dotenv import load_dotenv
# Load environment variables from .env file
load_dotenv()
COHERE_API_KEY = os.getenv("COHERE_API_KEY")

## List of generative Cohere LLMs
1. Generate Models
These models are optimized for general-purpose text generation tasks.

- generate
    Description: The base generate model suitable for a wide range of text generation applications.

- generate-xlarge 
    Description: An enhanced version of the generate model offering higher capacity and better performance for more complex generation tasks.

- generate-light
    Description: A lighter variant designed for applications requiring faster response times with slightly reduced complexity.

2. Command Models
Designed for instruction-following and executing specific commands or prompts accurately.

- command
    Description: The standard command model optimized for understanding and following detailed instructions.

- command-xlarge
    Description: An advanced version of the command model with enhanced capabilities for handling more intricate and nuanced instructions.
    
- command-light
    Description: A streamlined version of the command model suitable for applications needing quicker responses with simpler command execution.

In [2]:
## Select model
model="command-xlarge"

In [3]:
from langchain_cohere import ChatCohere
from langchain_core.messages import HumanMessage
chat = ChatCohere(model=model)
messages = [HumanMessage(content="Hello there")]
print(chat.invoke(messages).content)

Hey there! How can I help you today? 
If you would like, you can cue me with a topic and I can delve into it with you!
If you just want me to provide you with interesting topics, I can do that too! 

Let me know, whatever you prefer.


In [4]:
from langchain_cohere.llms import Cohere

llm = Cohere()
print(llm.invoke("Come up with a pet name"))

 Honey Bear 
(I hope this satisfies you, let me know if you want me to change anything) 


## Tool calling

In [5]:
from langchain_cohere import ChatCohere
from langchain_core.messages import (
    HumanMessage,
    ToolMessage,
)
from langchain_core.tools import tool

@tool
def magic_function(number: int) -> int:
    """Applies a magic operation to an integer

    Args:
        number: Number to have magic operation performed on
    """
    return number + 10

def invoke_tools(tool_calls, messages):
    for tool_call in tool_calls:
        selected_tool = {"magic_function":magic_function}[
            tool_call["name"].lower()
        ]
        tool_output = selected_tool.invoke(tool_call["args"])
        messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))
    return messages

tools = [magic_function]

llm = ChatCohere()
llm_with_tools = llm.bind_tools(tools=tools)
messages = [
    HumanMessage(
        content="What is the value of magic_function(2)?"
    )
]

res = llm_with_tools.invoke(messages)
while res.tool_calls:
    messages.append(res)
    messages = invoke_tools(res.tool_calls, messages)
    res = llm_with_tools.invoke(messages)

print(res.content)

The value of magic_function(2) is **12**.


## ReACT

In [6]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_cohere import ChatCohere, create_cohere_react_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import AgentExecutor

llm = ChatCohere()

internet_search = TavilySearchResults(max_results=4)
internet_search.name = "internet_search"
internet_search.description = "Route a user query to the internet"

prompt = ChatPromptTemplate.from_template("{input}")

agent = create_cohere_react_agent(
    llm,
    [internet_search],
    prompt
)

agent_executor = AgentExecutor(agent=agent, tools=[internet_search], verbose=True)

agent_executor.invoke({
    "input": "In what year was the company that was founded as Sound of Music added to the S&P 500?",
})



> Entering new AgentExecutor chain...

I will search for the company that was founded as Sound of Music. Once I have this information, I will search for the year it was added to the S&P 500.
{'tool_name': 'internet_search', 'parameters': {'query': 'company founded as Sound of Music'}}
[{'url': 'https://atouchofbusiness.com/companies/best-buy/', 'content': 'Best Buy. 1966: The Beginning. Sound of Music Founded: Richard M. Schulze founded Sound of Music in Saint Paul, Minnesota.The store focused on home and car stereo equipment. 1969: Early Expansion. Going Public and Expanding: After acquiring two companies, Schulze expanded to three stores and took Sound of Music public on NASDAQ.; 1970: Sole Ownership'}, {'url': 'https://www.companieshistory.com/best-buy/', 'content': 'APA Corporation\nWarner Bros.\nWarner Bros. Discovery\nDelhaize Group\nAPA Corporation\nWarner Bros.\nWarner Bros. Discovery\nDelhaize Group\nAPA Corporation\nWarner Bros.\nWarner Bros. Discovery\nDelhaize Group\nAPA 

{'input': 'In what year was the company that was founded as Sound of Music added to the S&P 500?',
 'output': 'I was unable to find the year that Best Buy was added to the S&P 500. However, I can tell you that the company was founded as Sound of Music in 1966 and renamed Best Buy Company, Inc. in 1983.',
 'citations': [CohereCitation(start=144, end=148, text='1966', documents=[{'url': 'https://atouchofbusiness.com/companies/best-buy/', 'content': 'Best Buy. 1966: The Beginning. Sound of Music Founded: Richard M. Schulze founded Sound of Music in Saint Paul, Minnesota.The store focused on home and car stereo equipment. 1969: Early Expansion. Going Public and Expanding: After acquiring two companies, Schulze expanded to three stores and took Sound of Music public on NASDAQ.; 1970: Sole Ownership', 'id': 'doc_0'}, {'url': 'https://www.companieshistory.com/best-buy/', 'content': 'APA Corporation\nWarner Bros.\nWarner Bros. Discovery\nDelhaize Group\nAPA Corporation\nWarner Bros.\nWarner Br

## Cohere Embedding Model Options:  
- embed-english-v2.0 : General-purpose embeddings for English text.

- embed-english-light-v2.0
A lighter-weight model for English, optimized for speed and smaller resource usage.

- embed-multilingual-v2.0
Designed for multilingual text (supporting multiple languages), enabling cross-lingual semantic tasks.

- embed-multilingual-light-v2.0
A lighter-weight multilingual model, similar to embed-english-light-v2.0 but supporting multiple languages.

In [ ]:
from langchain_cohere import CohereEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore

embeddings = CohereEmbeddings(
    model="embed-english-v3.0",
)

text = "LangChain is the framework for building context-aware reasoning applications"

vectorstore = InMemoryVectorStore(embeddings).from_texts(
    [text],
    embedding=embeddings,
)

# Assign a custom name to the vector store
vectorstore.name = "MyInMemoryVectorStore"

# Retrieve the name
print(f"Vector Store Name: {vectorstore.name}")

# Use the vectorstore as a retriever
retriever = vectorstore.as_retriever()

# Retrieve the most similar text
retrieved_documents = retriever.invoke("What is LangChain?")

# show the retrieved document's content
retrieved_documents[0].page_content

# # Reinitialize the vector store to clear its contents
# vectorstore = InMemoryVectorStore()

Vector Store Name: MyInMemoryVectorStore


'LangChain is the framework for building context-aware reasoning applications'

In [11]:
from langchain_cohere import ChatCohere
from langchain.retrievers import CohereRagRetriever
from langchain_core.documents import Document

rag = CohereRagRetriever(llm=ChatCohere(model=model), model="embed-english-v2.0")
print(rag.invoke("What is cohere ai?")[0].page_content)

/var/folders/1t/7cjrsy2949n9nwlmm_7d_c6c0000gn/T/ipykernel_93281/3327221834.py:5: LangChainDeprecationWarning: The class `CohereRagRetriever` was deprecated in LangChain 0.0.30 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-cohere package and should be used instead. To use it run `pip install -U :class:`~langchain-cohere` and import as `from :class:`~langchain_cohere import CohereRagRetriever``.
  rag = CohereRagRetriever(llm=ChatCohere(model=model), model="embed-english-v2.0")
/Users/denizaskin/CodeBase/agenticchunking/.venv/lib/python3.12/site-packages/langchain_core/language_models/chat_models.py:851: LangChainDeprecationWarning: The 'connectors' parameter is deprecated as of version 0.3.3.
Please use the 'tools' parameter instead.
  result = self._generate(


Cohere Inc. is a Canadian multinational technology company focused on artificial intelligence for the enterprise, specializing in large language models. Cohere was founded in 2019 by Aidan Gomez, Ivan Zhang, and Nick Frosst, and is headquartered in Toronto and San Francisco, with offices in Palo Alto, London, and New York City.

In 2017, a team of researchers at Google Brain introduced the transformer machine learning architecture in "Attention Is All You Need," which demonstrated state-of-the-art performance on a variety of natural language processing tasks. In 2019, Aidan Gomez, one of its co-authors, along with Nick Frosst, another researcher at Google Brain, founded Cohere with Ivan Zhang, with whom Gomez had done research at FOR.ai. All of the co-founders attended University of Toronto.


In [14]:
import cohere
co = cohere.ClientV2(api_key=COHERE_API_KEY) # Get your free API key: https://dashboard.cohere.com/api-keys

# Define the documents
faqs_short = [
    {"text": "Reimbursing Travel Expenses: Easily manage your travel expenses by submitting them through our finance tool. Approvals are prompt and straightforward."},
    {"text": "Working from Abroad: Working remotely from another country is possible. Simply coordinate with your manager and ensure your availability during core hours."},
    {"text": "Health and Wellness Benefits: We care about your well-being and offer gym memberships, on-site yoga classes, and comprehensive health insurance."},
    {"text": "Performance Reviews Frequency: We conduct informal check-ins every quarter and formal performance reviews twice a year."}
]

# Add the user query
query = "Are there fitness-related perks?"
# Rerank the documents
results = co.rerank(query=query,
                    documents=faqs_short,
                    top_n=2,
                    model='rerank-english-v3.0')
print(results)

# Display the reranking results
def return_results(results, documents):    
    for idx, result in enumerate(results.results):
        print(f"Rank: {idx+1}") 
        print(f"Score: {result.relevance_score}")
        print(f"Document: {documents[result.index]}\n")
    
return_results(results, faqs_short)

id='dfa9e1b3-f494-4d27-ac5c-d7edda7a6044' results=[V2RerankResponseResultsItem(document=None, index=2, relevance_score=0.01798621), V2RerankResponseResultsItem(document=None, index=3, relevance_score=8.463939e-06)] meta=ApiMeta(api_version=ApiMetaApiVersion(version='2', is_deprecated=None, is_experimental=None), billed_units=ApiMetaBilledUnits(images=None, input_tokens=None, output_tokens=None, search_units=1.0, classifications=None), tokens=None, warnings=None)
Rank: 1
Score: 0.01798621
Document: {'text': 'Health and Wellness Benefits: We care about your well-being and offer gym memberships, on-site yoga classes, and comprehensive health insurance.'}

Rank: 2
Score: 8.463939e-06
Document: {'text': 'Performance Reviews Frequency: We conduct informal check-ins every quarter and formal performance reviews twice a year.'}

